In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
import re
import string
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
import datetime

In [2]:
x_train = np.load('data/x_train.npy', allow_pickle=True)
x_test = np.load('data/x_test.npy', allow_pickle=True)
y_train = np.load('data/y_train.npy', allow_pickle=True)
y_test = np.load('data/y_test.npy', allow_pickle=True)

In [3]:
class TimeRecoder():
    start_time = 0
    end_time = 0
    
    def start(self):
        self.start_time = datetime.datetime.now()
        print('\n\n', '-'*10, ' START ', '-'*10, self.start_time, '\n\n')
        
    def end(self):
        self.end_time = datetime.datetime.now()
        print('\n\n', '-'*10, ' END ', '-'*10, self.end_time, '\n\n')
        self.checkTimeSpan()
        
    def checkTimeSpan(self):
        print('\n\n', '-'*10, ' TAKES ', '-'*10, self.end_time - self.start_time, '\n\n')

In [4]:
MAX_DOCUMENT_LENGTH = 100
N_FILTERS = 10
FILTER_SHAPE1 = [20, 256]
FILTER_SHAPE2 = [20, 1]
POOLING_WINDOW = 4
POOLING_STRIDE = 2
N_ANN_NODES = 60
MAX_LABEL = 15

no_epochs = 200
lr = 0.01

tf.reset_default_graph()

def char_cnn_model(x):
  
    input_layer = tf.reshape(
      tf.one_hot(x, 256), [-1, MAX_DOCUMENT_LENGTH, 256, 1])

    with tf.variable_scope('CNN_Layer1'):
        conv1 = tf.layers.conv2d(
            input_layer,
            filters=N_FILTERS,
            kernel_size=FILTER_SHAPE1,
            padding='VALID',
            activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(
            conv1,
            pool_size=POOLING_WINDOW,
            strides=POOLING_STRIDE,
            padding='SAME')
        
    with tf.variable_scope('CNN_Layer2'):
        conv2 = tf.layers.conv2d(
            pool1,
            filters=N_FILTERS,
            kernel_size=FILTER_SHAPE2,
            padding='VALID',
            activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(
            conv2,
            pool_size=POOLING_WINDOW,
            strides=POOLING_STRIDE,
            padding='SAME')

    dim = pool2.get_shape()[1].value * pool2.get_shape()[2].value * pool2.get_shape()[3].value 
    
    with tf.variable_scope('CNN_Flatten'):
        flatten = tf.reshape(pool2, [-1, dim])
    
    with tf.variable_scope('ANN'):
#         W1 = tf.Variable(tf.truncated_normal([dim, N_ANN_NODES], stddev=2.0/np.sqrt(dim)))
#         b1 = tf.Variable(tf.zeros([N_ANN_NODES]))
#         ann1 = tf.nn.relu(tf.matmul(flatten, W1) + b1)
        
#         W2 = tf.Variable(tf.truncated_normal([N_ANN_NODES, MAX_LABEL], stddev=1.0/np.sqrt(dim)))
#         b2 = tf.Variable(tf.zeros([MAX_LABEL]))
#         logits = tf.matmul(ann1, W2) + b2

        W1 = tf.Variable(tf.truncated_normal([dim, MAX_LABEL], stddev=1.0/np.sqrt(dim)))
        b1 = tf.Variable(tf.zeros([MAX_LABEL]))
        logits = tf.matmul(flatten, W1) + b1


    return input_layer, conv1, pool1, conv2, pool2, flatten, logits

# Create the model
x = tf.placeholder(tf.int64, [None, MAX_DOCUMENT_LENGTH])
y_ = tf.placeholder(tf.int64)

inputs, conv1, pool1, conv2, pool2, flatten, logits = char_cnn_model(x)

# Optimizer
entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.one_hot(y_, MAX_LABEL), logits=logits))
train_op = tf.train.AdamOptimizer(lr).minimize(entropy)

acc, acc_op = tf.metrics.accuracy(labels=y_, 
                                  predictions=tf.argmax(logits,1))

sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

print('input: ', sess.run([tf.shape(inputs)], {x: x_train, y_: y_train}))
print('conv1: ', sess.run([tf.shape(conv1)], {x: x_train, y_: y_train}))
print('pool1: ', sess.run([tf.shape(pool1)], {x: x_train, y_: y_train}))
print('conv2: ', sess.run([tf.shape(conv2)], {x: x_train, y_: y_train}))
print('pool2: ', sess.run([tf.shape(pool2)], {x: x_train, y_: y_train}))
print('flatten: ', sess.run([tf.shape(flatten)], {x: x_train, y_: y_train}))
# print('ann1: ', sess.run([tf.shape(ann1)], {x: x_train, y_: y_train}))
print('logits: ', sess.run([tf.shape(logits)], {x: x_train, y_: y_train}))

# print(sess.run([acc, acc_op]), {x: x_train, y_: y_train})
# print(sess.run([acc]), {x: x_train, y_: y_train})

entropy_on_training = []
accuracy_on_testing = []

timeRecoder = TimeRecoder()
timeRecoder.start()

for e in range(no_epochs):
    
    # training
    _, loss_  = sess.run([train_op, entropy], {x: x_train, y_: y_train})
    entropy_on_training.append(loss_)
    
    # testing
    predict = sess.run([logits], {x: x_test})
    accuracy_on_testing.append(accuracy_score(list(y_test), list(np.argmax(np.array(predict[0]), axis=1))))
    
     
    print('epoch %d: entropy: %f, accuracy: %f' % (e, entropy_on_training[-1], accuracy_on_testing[-1]))
    
timeRecoder.end()

Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
input:  [array([5600,  100,  256,    1], dtype=int32)]
conv1:  [array([5600,   81,    1,   10], dtype=int32)]
pool1:  [array([5600,   41,    1,   10], dtype=int32)]
conv2:  [array([5600,   22,    1,   10], dtype=int32)]
pool2:  [array([5600,   11,    1,   10], dtype=int32)]
flatten:  [array([5600,  110], dtype=int32)]
logits:  [array([5600,   15], dtype=int32)]


 ----------  START  ---------- 2019-11-13 08:30:37.134110 


epoch 0: entropy: 2.707174, accuracy: 0.071429
epoch 1: entropy: 2.676611, accuracy: 0.072857
epoch 2: entropy: 2.634583, accuracy: 0.097143
epoch 3: entropy: 2.611645, accuracy: 0.112857
epoch 4: entropy: 2.594865, accuracy: 0.175714
epoch 5: entropy: 2.513863, accuracy: 0.182857
epoch 6: entropy: 2.483295, accuracy: 0.217143
epoch 7: entropy: 2.420519, accuracy: 0.22

epoch 145: entropy: 0.438996, accuracy: 0.625714
epoch 146: entropy: 0.268400, accuracy: 0.584286
epoch 147: entropy: 0.427510, accuracy: 0.621429
epoch 148: entropy: 0.289576, accuracy: 0.627143
epoch 149: entropy: 0.315190, accuracy: 0.608571
epoch 150: entropy: 0.347382, accuracy: 0.617143
epoch 151: entropy: 0.268261, accuracy: 0.611429
epoch 152: entropy: 0.305527, accuracy: 0.618571
epoch 153: entropy: 0.307860, accuracy: 0.630000
epoch 154: entropy: 0.249254, accuracy: 0.611429
epoch 155: entropy: 0.290461, accuracy: 0.604286
epoch 156: entropy: 0.265345, accuracy: 0.617143
epoch 157: entropy: 0.246705, accuracy: 0.622857
epoch 158: entropy: 0.262093, accuracy: 0.618571
epoch 159: entropy: 0.253229, accuracy: 0.628571
epoch 160: entropy: 0.224060, accuracy: 0.604286
epoch 161: entropy: 0.251167, accuracy: 0.614286
epoch 162: entropy: 0.223676, accuracy: 0.621429
epoch 163: entropy: 0.219235, accuracy: 0.625714
epoch 164: entropy: 0.230356, accuracy: 0.624286
epoch 165: entropy: 

In [1]:
data = '''
epoch 0: entropy: 2.707174, accuracy: 0.071429
epoch 1: entropy: 2.676611, accuracy: 0.072857
epoch 2: entropy: 2.634583, accuracy: 0.097143
epoch 3: entropy: 2.611645, accuracy: 0.112857
epoch 4: entropy: 2.594865, accuracy: 0.175714
epoch 5: entropy: 2.513863, accuracy: 0.182857
epoch 6: entropy: 2.483295, accuracy: 0.217143
epoch 7: entropy: 2.420519, accuracy: 0.222857
epoch 8: entropy: 2.359095, accuracy: 0.202857
epoch 9: entropy: 2.303148, accuracy: 0.225714
epoch 10: entropy: 2.238824, accuracy: 0.258571
epoch 11: entropy: 2.168227, accuracy: 0.258571
epoch 12: entropy: 2.115414, accuracy: 0.288571
epoch 13: entropy: 2.074297, accuracy: 0.277143
epoch 14: entropy: 2.012943, accuracy: 0.292857
epoch 15: entropy: 1.977568, accuracy: 0.315714
epoch 16: entropy: 1.924968, accuracy: 0.328571
epoch 17: entropy: 1.851882, accuracy: 0.321429
epoch 18: entropy: 1.823469, accuracy: 0.345714
epoch 19: entropy: 1.762585, accuracy: 0.358571
epoch 20: entropy: 1.723281, accuracy: 0.372857
epoch 21: entropy: 1.682441, accuracy: 0.371429
epoch 22: entropy: 1.640038, accuracy: 0.388571
epoch 23: entropy: 1.586299, accuracy: 0.405714
epoch 24: entropy: 1.553366, accuracy: 0.387143
epoch 25: entropy: 1.518686, accuracy: 0.438571
epoch 26: entropy: 1.470957, accuracy: 0.454286
epoch 27: entropy: 1.442667, accuracy: 0.448571
epoch 28: entropy: 1.395328, accuracy: 0.454286
epoch 29: entropy: 1.349081, accuracy: 0.485714
epoch 30: entropy: 1.303161, accuracy: 0.477143
epoch 31: entropy: 1.265695, accuracy: 0.485714
epoch 32: entropy: 1.239038, accuracy: 0.511429
epoch 33: entropy: 1.220249, accuracy: 0.494286
epoch 34: entropy: 1.224847, accuracy: 0.500000
epoch 35: entropy: 1.202622, accuracy: 0.502857
epoch 36: entropy: 1.142559, accuracy: 0.520000
epoch 37: entropy: 1.100663, accuracy: 0.510000
epoch 38: entropy: 1.105683, accuracy: 0.524286
epoch 39: entropy: 1.084880, accuracy: 0.547143
epoch 40: entropy: 1.015519, accuracy: 0.525714
epoch 41: entropy: 1.045610, accuracy: 0.518571
epoch 42: entropy: 1.007403, accuracy: 0.558571
epoch 43: entropy: 0.948354, accuracy: 0.551429
epoch 44: entropy: 0.977390, accuracy: 0.580000
epoch 45: entropy: 0.908772, accuracy: 0.558571
epoch 46: entropy: 0.914359, accuracy: 0.578571
epoch 47: entropy: 0.871870, accuracy: 0.574286
epoch 48: entropy: 0.869602, accuracy: 0.560000
epoch 49: entropy: 0.851917, accuracy: 0.575714
epoch 50: entropy: 0.817484, accuracy: 0.580000
epoch 51: entropy: 0.826538, accuracy: 0.584286
epoch 52: entropy: 0.786587, accuracy: 0.577143
epoch 53: entropy: 0.805238, accuracy: 0.577143
epoch 54: entropy: 0.806951, accuracy: 0.567143
epoch 55: entropy: 0.846887, accuracy: 0.567143
epoch 56: entropy: 0.781877, accuracy: 0.581429
epoch 57: entropy: 0.723016, accuracy: 0.590000
epoch 58: entropy: 0.769715, accuracy: 0.590000
epoch 59: entropy: 0.733493, accuracy: 0.590000
epoch 60: entropy: 0.701744, accuracy: 0.600000
epoch 61: entropy: 0.695943, accuracy: 0.592857
epoch 62: entropy: 0.702045, accuracy: 0.607143
epoch 63: entropy: 0.651304, accuracy: 0.602857
epoch 64: entropy: 0.660356, accuracy: 0.598571
epoch 65: entropy: 0.650742, accuracy: 0.602857
epoch 66: entropy: 0.618177, accuracy: 0.592857
epoch 67: entropy: 0.628473, accuracy: 0.607143
epoch 68: entropy: 0.607117, accuracy: 0.612857
epoch 69: entropy: 0.593208, accuracy: 0.612857
epoch 70: entropy: 0.592252, accuracy: 0.608571
epoch 71: entropy: 0.580704, accuracy: 0.615714
epoch 72: entropy: 0.565271, accuracy: 0.615714
epoch 73: entropy: 0.558468, accuracy: 0.617143
epoch 74: entropy: 0.554573, accuracy: 0.610000
epoch 75: entropy: 0.536868, accuracy: 0.615714
epoch 76: entropy: 0.529213, accuracy: 0.615714
epoch 77: entropy: 0.528875, accuracy: 0.610000
epoch 78: entropy: 0.513613, accuracy: 0.617143
epoch 79: entropy: 0.501910, accuracy: 0.622857
epoch 80: entropy: 0.500522, accuracy: 0.615714
epoch 81: entropy: 0.495108, accuracy: 0.615714
epoch 82: entropy: 0.489804, accuracy: 0.617143
epoch 83: entropy: 0.495441, accuracy: 0.611429
epoch 84: entropy: 0.511759, accuracy: 0.592857
epoch 85: entropy: 0.573876, accuracy: 0.597143
epoch 86: entropy: 0.589311, accuracy: 0.605714
epoch 87: entropy: 0.525472, accuracy: 0.622857
epoch 88: entropy: 0.460804, accuracy: 0.607143
epoch 89: entropy: 0.503671, accuracy: 0.601429
epoch 90: entropy: 0.521689, accuracy: 0.610000
epoch 91: entropy: 0.454571, accuracy: 0.618571
epoch 92: entropy: 0.453258, accuracy: 0.608571
epoch 93: entropy: 0.470767, accuracy: 0.614286
epoch 94: entropy: 0.448163, accuracy: 0.622857
epoch 95: entropy: 0.435946, accuracy: 0.617143
epoch 96: entropy: 0.427217, accuracy: 0.614286
epoch 97: entropy: 0.429498, accuracy: 0.622857
epoch 98: entropy: 0.405900, accuracy: 0.622857
epoch 99: entropy: 0.409564, accuracy: 0.615714
epoch 100: entropy: 0.408733, accuracy: 0.622857
epoch 101: entropy: 0.380417, accuracy: 0.627143
epoch 102: entropy: 0.400595, accuracy: 0.618571
epoch 103: entropy: 0.384293, accuracy: 0.622857
epoch 104: entropy: 0.368417, accuracy: 0.630000
epoch 105: entropy: 0.383931, accuracy: 0.627143
epoch 106: entropy: 0.364572, accuracy: 0.627143
epoch 107: entropy: 0.355596, accuracy: 0.620000
epoch 108: entropy: 0.363590, accuracy: 0.625714
epoch 109: entropy: 0.348012, accuracy: 0.620000
epoch 110: entropy: 0.339004, accuracy: 0.620000
epoch 111: entropy: 0.344978, accuracy: 0.625714
epoch 112: entropy: 0.334212, accuracy: 0.624286
epoch 113: entropy: 0.326188, accuracy: 0.620000
epoch 114: entropy: 0.326764, accuracy: 0.631429
epoch 115: entropy: 0.319251, accuracy: 0.624286
epoch 116: entropy: 0.316500, accuracy: 0.621429
epoch 117: entropy: 0.310245, accuracy: 0.625714
epoch 118: entropy: 0.304958, accuracy: 0.625714
epoch 119: entropy: 0.305843, accuracy: 0.622857
epoch 120: entropy: 0.295546, accuracy: 0.631429
epoch 121: entropy: 0.289104, accuracy: 0.624286
epoch 122: entropy: 0.291174, accuracy: 0.624286
epoch 123: entropy: 0.284987, accuracy: 0.627143
epoch 124: entropy: 0.277974, accuracy: 0.628571
epoch 125: entropy: 0.275706, accuracy: 0.622857
epoch 126: entropy: 0.272447, accuracy: 0.628571
epoch 127: entropy: 0.268867, accuracy: 0.632857
epoch 128: entropy: 0.264994, accuracy: 0.622857
epoch 129: entropy: 0.260791, accuracy: 0.624286
epoch 130: entropy: 0.257708, accuracy: 0.638571
epoch 131: entropy: 0.256193, accuracy: 0.627143
epoch 132: entropy: 0.258906, accuracy: 0.620000
epoch 133: entropy: 0.269896, accuracy: 0.615714
epoch 134: entropy: 0.305130, accuracy: 0.607143
epoch 135: entropy: 0.406387, accuracy: 0.554286
epoch 136: entropy: 0.689787, accuracy: 0.535714
epoch 137: entropy: 1.010723, accuracy: 0.575714
epoch 138: entropy: 0.556296, accuracy: 0.564286
epoch 139: entropy: 0.573826, accuracy: 0.578571
epoch 140: entropy: 0.563458, accuracy: 0.608571
epoch 141: entropy: 0.425406, accuracy: 0.575714
epoch 142: entropy: 0.491839, accuracy: 0.598571
epoch 143: entropy: 0.376070, accuracy: 0.615714
epoch 144: entropy: 0.367106, accuracy: 0.615714
epoch 145: entropy: 0.438996, accuracy: 0.625714
epoch 146: entropy: 0.268400, accuracy: 0.584286
epoch 147: entropy: 0.427510, accuracy: 0.621429
epoch 148: entropy: 0.289576, accuracy: 0.627143
epoch 149: entropy: 0.315190, accuracy: 0.608571
epoch 150: entropy: 0.347382, accuracy: 0.617143
epoch 151: entropy: 0.268261, accuracy: 0.611429
epoch 152: entropy: 0.305527, accuracy: 0.618571
epoch 153: entropy: 0.307860, accuracy: 0.630000
epoch 154: entropy: 0.249254, accuracy: 0.611429
epoch 155: entropy: 0.290461, accuracy: 0.604286
epoch 156: entropy: 0.265345, accuracy: 0.617143
epoch 157: entropy: 0.246705, accuracy: 0.622857
epoch 158: entropy: 0.262093, accuracy: 0.618571
epoch 159: entropy: 0.253229, accuracy: 0.628571
epoch 160: entropy: 0.224060, accuracy: 0.604286
epoch 161: entropy: 0.251167, accuracy: 0.614286
epoch 162: entropy: 0.223676, accuracy: 0.621429
epoch 163: entropy: 0.219235, accuracy: 0.625714
epoch 164: entropy: 0.230356, accuracy: 0.624286
epoch 165: entropy: 0.208442, accuracy: 0.615714
epoch 166: entropy: 0.209808, accuracy: 0.612857
epoch 167: entropy: 0.212943, accuracy: 0.625714
epoch 168: entropy: 0.198087, accuracy: 0.618571
epoch 169: entropy: 0.201264, accuracy: 0.618571
epoch 170: entropy: 0.198937, accuracy: 0.617143
epoch 171: entropy: 0.186771, accuracy: 0.620000
epoch 172: entropy: 0.194171, accuracy: 0.618571
epoch 173: entropy: 0.185114, accuracy: 0.614286
epoch 174: entropy: 0.182246, accuracy: 0.621429
epoch 175: entropy: 0.184230, accuracy: 0.617143
epoch 176: entropy: 0.174724, accuracy: 0.617143
epoch 177: entropy: 0.176949, accuracy: 0.611429
epoch 178: entropy: 0.172873, accuracy: 0.621429
epoch 179: entropy: 0.169083, accuracy: 0.618571
epoch 180: entropy: 0.170041, accuracy: 0.618571
epoch 181: entropy: 0.164402, accuracy: 0.612857
epoch 182: entropy: 0.163716, accuracy: 0.610000
epoch 183: entropy: 0.161547, accuracy: 0.618571
epoch 184: entropy: 0.157837, accuracy: 0.614286
epoch 185: entropy: 0.158163, accuracy: 0.612857
epoch 186: entropy: 0.153952, accuracy: 0.611429
epoch 187: entropy: 0.153135, accuracy: 0.607143
epoch 188: entropy: 0.151048, accuracy: 0.612857
epoch 189: entropy: 0.148116, accuracy: 0.617143
epoch 190: entropy: 0.147811, accuracy: 0.610000
epoch 191: entropy: 0.144171, accuracy: 0.608571
epoch 192: entropy: 0.143933, accuracy: 0.612857
epoch 193: entropy: 0.141180, accuracy: 0.614286
epoch 194: entropy: 0.139653, accuracy: 0.605714
epoch 195: entropy: 0.138221, accuracy: 0.611429
epoch 196: entropy: 0.135824, accuracy: 0.608571
epoch 197: entropy: 0.134997, accuracy: 0.604286
epoch 198: entropy: 0.132621, accuracy: 0.612857
epoch 199: entropy: 0.131517, accuracy: 0.612857'''

In [5]:
data.split('\n')[1].split(' ')

['epoch', '0:', 'entropy:', '2.707174,', 'accuracy:', '0.071429']

In [6]:
e = []
a = []

for row in data.split('\n')[1:]:
    e.append(float(row.split(' ')[3][:-1]))
    a.append(float(row.split(' ')[-1]))

In [10]:
import numpy as np
import os
np.save(os.path.join('other', 'npy', 'entropy_on_training_q1.npy'), np.array(e))
np.save(os.path.join('other', 'npy', 'accuracy_on_testing_q1.npy'), np.array(a))